In [10]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Reshape, Permute, Multiply, Input, Activation
from keras.models import Sequential, Model
import tensorflow.keras.backend as K
import numpy as np
import cv2
import matplotlib.cm as cm

In [11]:
def SqueezeExcite(x, ratio=16, name=''):
    nb_chan = K.int_shape(x)[-1]

    y = GlobalAveragePooling2D(name='SE_avg_{}'.format(name))(x)
    y = Dense(nb_chan // ratio, activation='relu', name='se_dense1_{}'.format(name))(y)
    y = Dense(nb_chan, activation='sigmoid', name='dense2_{}'.format(name))(y)

    y = Multiply(name='se_mul_{}'.format(name))([x, y])
    return y

def SE_Conv2D_block(x, filters, kernel_size, name):
    y = Conv2D(filters, kernel_size, padding='same', name='conv_{}'.format(name))(x)
    #squeeze excite in this
    y = SqueezeExcite(y, ratio=16, name=name)
    #other relu and maxpools
    y = BatchNormalization(name='BatchNorm_{}'.format(name))(y)
    y = Activation('relu', name='Activation_{}'.format(name))(y)
    y = MaxPool2D(pool_size=(2, 2), padding='same', name='{}_pool'.format(name))(y)
    y = Dropout(0.25)(y)
    return y


def SE_Dense_block(x, size, name, bn=True):
    y = Dense(size, name='dense_{}'.format(name))(x)
    if bn:
        y = BatchNormalization(name='BatchNorm_{}'.format(name))(y)
    y = Activation('relu', name='Activation_{}'.format(name))(y)
    return y

In [12]:
#huge thanks to -->https://github.com/arthurdouillard/keras-squeeze_and_excitation_network


def SE_Model(nb_class, input_shape, include_top=True, weights = False):
    """SE net without the splitted stream."""

    img_input = Input(shape=input_shape)

    x = SE_Conv2D_block(img_input, 32, (3, 3), name='block1')
    

    params = [
        (64, (3, 3)),
        (128, (3, 3)),
        (256, (3, 3))
    ]

    for i, (filters, kernel_size) in enumerate(params, start=2):
        x = SE_Conv2D_block(x, filters, kernel_size, name='block{}'.format(i))

    if include_top:
        x = Flatten(name='flatten')(x)
#         x = SE_Dense_block(x, 512, 'top1', bn=True)
#         x = SE_Dense_block(x, 128, 'top2', bn=True)
#         x = SE_Dense_block(x, 32, 'top3', bn=True)
        x = Dense(nb_class, activation='sigmoid')(x)

    model = Model(inputs=img_input, outputs=x)

    if weights:
        print('Loading')
        model.load_weights(weights)
        print("Weight Loaded")

    return model

model = SE_Model(nb_class=1, input_shape=(128, 128, 3), weights='SEModelWeights.h5')
model.summary()
# model.compile()

optimizer = keras.optimizers.Adam(lr=0.00001)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()
val_loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

Loading
Weight Loaded
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv_block1 (Conv2D)            (None, 128, 128, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
SE_avg_block1 (GlobalAveragePoo (None, 32)           0           conv_block1[0][0]                
__________________________________________________________________________________________________
se_dense1_block1 (Dense)        (None, 2)            66          SE_avg_block1[0][0]              
______________________________________________________________________

In [13]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)
    return val_preds

In [14]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [15]:
def display_gradcam(img, heatmap, alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img('img.jpg')
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    
    
    superimposed_img.save('img.jpg')
    return superimposed_img

    # Display Grad CAM

In [16]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=[gray,gray,gray]
    to_test = np.resize(gray,(1,128,128,3))
#     print(to_test.shape)
    result = test_step(to_test)
    result = np.reshape(tf.get_static_value(result),(1,1))[0][0]
    
    # Display the resulting frame
    prediction = ''
    if result>0.5:
        prediction = 'non-drowsy'
    else:
        prediction = 'drowsy'
    
    cv2.imwrite('img.jpg', frame)
    heatmap = make_gradcam_heatmap(tf.expand_dims(np.resize(gray,(128,128,3)),0), model, 'conv_block4')
    frame = display_gradcam(np.resize(frame,(128,128,3)), heatmap)
    frame = np.array(frame)[:, :, ::-1].copy()
    cv2.putText(frame, str(result), (10, 160), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.putText(frame, prediction, (10, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()